In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# Transformasi dataset dengan beberapa langkah preprocessing
transform = transforms.Compose([
    # Mengubah gambar menjadi format tensor
    transforms.ToTensor(),
    # Normalisasi nilai pixel gambar ke rentang [-1, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Load CIFAR-10 Dataset
batch_size = 32
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:02<00:00, 76.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Membagi dataset pelatihan menjadi subset pelatihan dan validasi
train_size = int(0.8 * len(train_dataset))  # 80% data untuk pelatihan
val_size = len(train_dataset) - train_size  # Sisanya 20% untuk validasi
train_data, val_data = random_split(train_dataset, [train_size, val_size])  # Membagi dataset

# Membuat DataLoader untuk subset pelatihan, validasi, dan pengujian
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)  # DataLoader untuk pelatihan
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)     # DataLoader untuk validasi
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # DataLoader untuk pengujian

In [5]:
# CNN Arsitektur
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)

        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Fungsi untuk Melatih dan Mengevaluasi Model
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs, early_stopping_patience=10):
    # Menentukan perangkat (GPU atau CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Inisialisasi model CNN dengan parameter kernel_size dan pooling_type
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Pemilihan optimizer berdasarkan jenis yang ditentukan
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Scheduler untuk menurunkan learning rate jika tidak ada perbaikan
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)

    # Variabel untuk melacak loss terbaik dan jumlah epoch tanpa peningkatan
    best_loss = float('inf')
    patience_counter = 0

    print(f"Evaluating with params: {{'epochs': {epochs}, 'kernel_size': {kernel_size}, 'optimizer_type': '{optimizer_type}', 'pooling_type': '{pooling_type}'}}")

    # Loop untuk setiap epoch
    for epoch in range(epochs):
        # Mode pelatihan
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Reset gradien, lakukan forward dan backward pass, serta update parameter
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Rata-rata loss untuk data pelatihan
        train_loss = running_loss / len(train_loader)

        # Mode evaluasi untuk validasi
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        # Menampilkan hasil pada setiap epoch
        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Update scheduler dengan validasi loss
        scheduler.step(val_loss)

        # Early Stopping jika tidak ada perbaikan pada validasi loss
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

    # Evaluasi akurasi pada dataset pengujian
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Menghitung akurasi
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Melakukan eksperimen dengan berbagai kombinasi parameter
kernel_sizes = [3, 5, 7]         # Daftar ukuran kernel
pooling_types = ['max', 'avg']   # Jenis pooling
optimizers = ['SGD', 'RMSProp', 'Adam']  # Jenis optimizer
epoch_list = [5, 50, 100, 250, 350]  # Daftar jumlah epoch

results = []  # Untuk menyimpan hasil eksperimen

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epoch_list:
                print(f"Running experiment with kernel_size={kernel_size}, pooling_type={pooling_type}, optimizer={optimizer}, epochs={epochs}")
                accuracy = train_and_evaluate(kernel_size, pooling_type, optimizer, epochs, early_stopping_patience=10)
                results.append((kernel_size, pooling_type, optimizer, epochs, accuracy))

# Menampilkan hasil akhir eksperimen
print("\nHasil Akhir Eksperimen:")
for result in results:
    print(f"Kernel Size: {result[0]}, Pooling: {result[1]}, Optimizer: {result[2]}, Epochs: {result[3]}, Accuracy: {result[4]:.2f}%")

Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=5
Evaluating with params: {'epochs': 5, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5, Loss: 1.6387, Val Loss: 1.2828
Epoch 2/5, Loss: 1.2530, Val Loss: 1.0405
Epoch 3/5, Loss: 1.0911, Val Loss: 1.0781
Epoch 4/5, Loss: 0.9806, Val Loss: 0.9340
Epoch 5/5, Loss: 0.8986, Val Loss: 0.9010
Accuracy: 68.89%
Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=50
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 1.6443, Val Loss: 1.3740
Epoch 2/50, Loss: 1.2381, Val Loss: 1.0460
Epoch 3/50, Loss: 1.0819, Val Loss: 0.9470
Epoch 4/50, Loss: 0.9686, Val Loss: 0.9245
Epoch 5/50, Loss: 0.8884, Val Loss: 0.9631
Epoch 6/50, Loss: 0.8291, Val Loss: 0.9018
Epoch 7/50, Loss: 0.7569, Val Loss: 0.8820
Epoch 8/50, Loss: 0.7099, Val Loss: 0.8678
Epoch 9/50, Loss: 0.6610, Val Loss: 0.8753
Epoch 10/50, Loss: 0.6259, Val Loss: 0.9181
Epoch 11/50, Loss: 0.5891, Val Loss: 0.9086
Epoch 12/50, Loss: 0.5659, Val Loss: 0.9750
Epoch 13/50, Loss: 0.5393, Val Loss: 0.9788
Epoch 14/50, Loss: 0.50